In [1]:
from bitmap_impl.mcelice_bitmap import BitMatrix, BitVector
import re


def load_text_as_bitmap_vector(path: str) -> BitVector:
    """
    Lädt numerische Daten aus einer Textdatei und gibt einen BitVector zurück.

    Automatische Formaterkennung:
    - Leerzeichen-/Tab-getrennte Zahlen
    - CSV (Komma-getrennt)
    - Reine Bitfolge (0/1) ohne Trennzeichen
    - Mit Klammern umgeben: (...) oder [...]

    Parameter:
    - path: Pfad zur Datei (z.B. "ciphertext.txt")

    Rückgabe:
    - BitVector mit den geladenen Werten
    """
    try:
        # Versuche Whitespace-getrennte Zahlen zu laden
        with open(path, "r", encoding="utf-8") as f:
            content = f.read().strip()
        
        # Entferne umschließende Klammern falls vorhanden
        content = content.strip()
        if (content.startswith('(') and content.endswith(')')) or \
           (content.startswith('[') and content.endswith(']')):
            content = content[1:-1].strip()
        
        # Extrahiere nur Ziffern (0 und 1 für Binärdaten)
        # Verwende regex um alle Ziffern zu finden
        digits = [int(x) for x in re.findall(r'\d+', content)]
        
        if len(digits) == 0:
            raise ValueError(f"Konnte keine numerischen Daten aus {path} lesen.")
        
        return BitVector(len(digits), digits)
    
    except Exception as e:
        raise ValueError(f"Fehler beim Laden von {path}: {e}")


def load_text_as_bitmap_matrix(path: str, n: int = None) -> BitMatrix:
    """
    Lädt numerische Daten aus einer Textdatei und gibt eine BitMatrix zurück.

    Automatische Formaterkennung:
    - Mehrzeilige Matrix mit [ ] Klammern pro Zeile
    - Fortsetzungszeilen (eingerückt)
    - Flache Liste, die zu Matrix umgeformt wird (wenn n gegeben ist)

    Parameter:
    - path: Pfad zur Datei (z.B. "public_key.txt")
    - n: Optional - Anzahl Spalten für Reshape von flachen Daten

    Rückgabe:
    - BitMatrix mit den geladenen Werten
    """
    try:
        with open(path, "r", encoding="utf-8") as f:
            lines = f.readlines()
        
        # Parse Matrix: Sammle Zeilen die mit [ beginnen
        matrix_data = []
        current_row = []
        
        for line in lines:
            line = line.strip()
            if not line:
                continue
            
            # Neue Matrix-Zeile beginnt mit [
            if line.startswith('['):
                # Speichere vorherige Zeile falls vorhanden
                if current_row:
                    matrix_data.append(current_row)
                # Starte neue Zeile
                current_row = []
                # Entferne [ und ]
                line = line.replace('[', '').replace(']', '')
            else:
                # Fortsetzungszeile - entferne nur ]
                line = line.replace(']', '')
            
            # Extrahiere Zahlen aus dieser Zeile
            numbers = [int(x) for x in re.findall(r'\d+', line)]
            current_row.extend(numbers)
        
        # Speichere letzte Zeile
        if current_row:
            matrix_data.append(current_row)
        
        # Falls wir Matrix-Zeilen gefunden haben
        if matrix_data and len(matrix_data) > 1:
            rows = len(matrix_data)
            cols = len(matrix_data[0]) if matrix_data else 0
            
            # Validiere dass alle Zeilen gleich lang sind
            if not all(len(row) == cols for row in matrix_data):
                raise ValueError("Nicht alle Matrix-Zeilen haben die gleiche Länge")
            
            return BitMatrix(rows, cols, matrix_data)
        
        # Fallback: Flache Liste
        if matrix_data:
            digits = matrix_data[0] if len(matrix_data) == 1 else sum(matrix_data, [])
        else:
            # Lese gesamten Inhalt und extrahiere alle Zahlen
            with open(path, "r", encoding="utf-8") as f:
                content = f.read()
            digits = [int(x) for x in re.findall(r'\d+', content)]
        
        if n is not None and len(digits) % n == 0:
            # Reshape zu Matrix
            k = len(digits) // n
            matrix_data = []
            for i in range(k):
                row = digits[i * n:(i + 1) * n]
                matrix_data.append(row)
            return BitMatrix(k, n, matrix_data)
        else:
            # Als einzelne Zeile behandeln
            return BitMatrix(1, len(digits), [digits])
    
    except Exception as e:
        raise ValueError(f"Fehler beim Laden von {path}: {e}")


def load_ciphertext_and_public_key_bitmap(
    ciphertext_path: str = "ciphertext.txt",
    public_key_path: str = "public_key.txt",
):
    """
    Lädt `ciphertext` und `public_key` aus Textdateien als Bitmap-Objekte.

    - Ciphertext wird als BitVector geladen
    - Public Key wird als BitMatrix geladen (mit automatischem Reshape basierend auf n)

    Rückgabe:
    - (BitVector, BitMatrix) - ciphertext und public_key
    """
    # Lade Ciphertext
    ciphertext = load_text_as_bitmap_vector(ciphertext_path)
    n = ciphertext.length
    
    # Lade Public Key (mit n für automatisches Reshape)
    public_key = load_text_as_bitmap_matrix(public_key_path, n=n)
    
    return ciphertext, public_key

In [2]:
# Laden und Formen prüfen mit Bitmaps
ciphertext, public_key = load_ciphertext_and_public_key_bitmap()
print("ciphertext Länge:", ciphertext.length)
print("ciphertext Daten:", ciphertext.data[:20], "...")  # Zeige erste 20 Bits
print("public_key Größe:", f"{public_key.rows}x{public_key.cols}")
print("public_key erste Zeile:", public_key.to_list()[0] if public_key.rows > 0 else [])

ciphertext Länge: 167
ciphertext Daten: [1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1] ...
public_key Größe: 57x167
public_key erste Zeile: [0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0]


# Angriff mit Information Set Decoding

In [5]:
from bitmap_impl.isd_attack import InformationSetDecoding

# Initialisiere den Angriff
isd = InformationSetDecoding()
isd.set_G_pub(public_key)

# Führe den Angriff aus
print("Starte Information Set Decoding Angriff...")
decoded = isd.attack(ciphertext, t=11, max_attempts=100000, verbose=True)

if decoded:
    print("\n✓ Dekodierte Nachricht:", decoded.to_list())
    print("Als String:", ''.join(map(str, decoded.to_list())))
else:
    print("\n✗ Angriff fehlgeschlagen")

Starte Information Set Decoding Angriff...
Erwarte 11 Fehler in der Übertragung
Geschätzte Anzahl der Versuche: 1565.971538151941
✓ Erfolg nach 174 Versuchen!
Median Zeit pro Versuch: 0.007100 Sekunden
Geschätzte Zeit laut geschätzten Versuchen * Median-Zeit: 11.12 Sekunden
Tatsächliche Zeit: 0.42 Sekunden

✓ Dekodierte Nachricht: [1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0]
Als String: 110111001101101010011010101000101101000100100000110001010


# Angriff mit Lee Brickel Verallgemeinerung von ISD

In [6]:
from bitmap_impl.lee_brickel_attack import LeeBrickellAttack

# Initialisiere den Angriff
lba = LeeBrickellAttack()
lba.set_G_pub(public_key)

print("Starte Lee-Brickel Angriff...")
decoded_lba = lba.attack(ciphertext, t=11, p=0, max_attempts=100000, verbose=True)

if decoded_lba:
    print("\n✓ Dekodierte Nachricht (Lee-Brickel):", decoded_lba.to_list())
    print("Als String:", ''.join(map(str, decoded_lba.to_list())))
else:
    print("\n✗ Lee-Brickel Angriff fehlgeschlagen")
 

Starte Lee-Brickel Angriff...
✓ Erfolg nach 256 Informationssets und 70 Checks!
  Fehlervektorgewicht im Information Set: 0

✓ Dekodierte Nachricht (Lee-Brickel): [1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0]
Als String: 110111001101101010011010101000101101000100100000110001010
